# Agents
代理的核心思想是使用语言模型来选择要采取的一系列行动。在链中，一系列动作是硬编码的(用代码)。在代理中，语言模型被用作推理引擎，以确定以何种顺序采取哪些操作。

In [12]:
!pip install python-dotenv -U

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
  Using cached https://mirrors.aliyun.com/pypi/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 0.21.1
    Uninstalling python-dotenv-0.21.1:
      Successfully uninstalled python-dotenv-0.21.1


In [18]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

## Quickstart
要快速开始与Agents，请查看这个入门指南。这包括初始化Agent、创建工具和添加内存等基础知识。

为了更好地理解Agent框架，让我们构建一个具有两个工具的代理:一个用于在线查找，另一个用于查找我们加载到索引中的特定数据。

这将假设你有LLM和检索的知识，所以如果你还没有研究过这些部分，建议你这样做。

### Setup: LangSmith

根据定义，Agent在返回面向用户的输出之前，采取一个自确定的、依赖于输入的步骤序列。这使得调试这些系统特别棘手，可观察性特别重要。LangSmith在这种情况下特别有用。

当使用LangChain构建时，所有步骤都会自动在LangSmith中进行跟踪。要设置LangSmith，我们只需要设置以下环境变量：

```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="<your-api-key>"
```

### 定义工具
我们首先需要创建我们想要使用的工具。我们将使用两个工具:Tavily(用于在线搜索)，然后是我们将创建的本地索引的检索器
#### Tavily
我们在LangChain中有一个内置的工具，可以轻松地使用Tavily搜索引擎作为工具。注意，这需要一个API密钥——它们有一个免费的层，但是如果你没有一个或者不想创建一个，你可以忽略这个步骤。


In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [20]:
search = TavilySearchResults()
search.invoke("what is the weather in SF")

"HTTPError('502 Server Error: Bad Gateway for url: https://api.tavily.com/search')"

### Retriever
我们还将在我们自己的一些数据上创建一个检索器。有关这里每个步骤的更深入解释，请参阅本节。

In [21]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [22]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";co

In [23]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

### Tools
现在我们已经创建了这两个工具，我们可以创建一个将在下游使用的工具列表。

In [28]:
tools = [search, retriever_tool]

#### Create the agent
既然已经定义了工具，就可以创建代理了。我们将使用OpenAI Functions代理—有关这种类型的代理以及其他选项的更多信息，请参阅本指南。

首先，我们选择我们想要引导代理的LLM。


In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

接下来，我们选择要用来引导代理的提示符。如果您想查看此提示的内容并访问LangSmith，可以转到

In [26]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

现在，我们可以用LLM、提示符和工具初始化Agent。代理负责接收输入并决定采取什么行动。至关重要的是，Agent不执行这些操作——这是由AgentExecutor完成的(下一步)。有关如何考虑这些组件的更多信息，请参阅[我们的概念指南](https://python.langchain.com/v0.1/docs/modules/agents/concepts/)。

In [29]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

最后，我们将Agent(大脑)与AgentExecutor内部的工具(它将重复调用代理和执行工具)结合起来。

In [30]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Run the agent
现在我们可以在几个查询上运行代理了!请注意，目前这些都是无状态查询(它不会记住以前的交互)。

In [31]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [32]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Lo

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith can help with testing by providing a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, ensuring that you can ship quickly and with confidence. Here are some steps to get started with LangSmith for testing purposes:\n\n1. Install LangSmith using Python or TypeScript:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Export the necessary environment variables like `LANGCHAIN_TRACING_V2`, `LANGCHAIN_API_KEY`, and `OPENAI_API_KEY`.\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can log your first trace to start testing your application.\n\nBy following these steps and utilizing LangSmith's features, you can effectively test and 

In [33]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


HTTPError('502 Server Error: Bad Gateway for url: https://api.tavily.com/search')
Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`
responded: I encountered an issue while trying to retrieve the weather information for San Francisco. Let me try again.

HTTPError('502 Server Error: Bad Gateway for url: https://api.tavily.com/search')I apologize for the inconvenience, but it seems there is a technical issue preventing me from retrieving the weather information for San Francisco at the moment. Is there anything else I can assist you with?

> Finished chain.


{'input': 'whats the weather in sf?',
 'output': 'I apologize for the inconvenience, but it seems there is a technical issue preventing me from retrieving the weather information for San Francisco at the moment. Is there anything else I can assist you with?'}

### Adding in memory
如前所述，该代理是无状态的。这意味着它不记得以前的交互。为了给它内存，我们需要传递之前的聊天记录。注意:由于我们使用的提示符，它需要被称为聊天记录。如果我们使用不同的提示符，我们可以更改变量名

In [34]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [35]:
from langchain_core.messages import AIMessage, HumanMessage

In [36]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...
Your name is Bob. How can I help you, Bob?

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob. How can I help you, Bob?'}

如果我们想要自动跟踪这些消息，我们可以将其封装在   `RunnableWithMessageHistory`中。有关如何使用它的更多信息，请参阅本指南。

In [37]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
message_history = ChatMessageHistory()

In [38]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [39]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [40]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Your name is Bob! How can I assist you, Bob?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob! How can I assist you, Bob?'}

## Custom agent
本手册将介绍如何创建您自己的自定义Agent。
在这个例子中，我们将使用OpenAI工具调用来创建这个代理。这通常是创建代理的最可靠的方法。我们将首先创建它没有内存，但我们将展示如何添加内存。对话需要记忆。

### Load the LLM
首先，让我们加载将要用来控制Agent的语言模型。


In [41]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### Define Tools

接下来，让我们定义一些要使用的工具。让我们编写一个非常简单的Python函数来计算传入的单词的长度

注意，这里我们使用的函数docstring非常重要。在这里阅读更多关于为什么会出现这种情况的信息

In [42]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [48]:
tools = [get_word_length]

### Create Prompt
现在让我们创建提示符。因为OpenAI函数调用针对工具使用进行了微调，所以我们几乎不需要任何关于如何推理或如何输出格式的说明。我们只有两个输入变量`:input`和`agent_scratchpad.input`应该是包含用户目标的字符串。`agent_scratchpad`是一个消息序列，其中包含以前的Agent工具调用和相应的工具输出。

In [49]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

### Bind tools to LLM
Agent如何知道它可以使用哪些工具

在这种情况下，我们依赖于OpenAI工具调用llm，它将工具作为一个单独的参数，并经过专门训练，知道何时调用这些工具。

为了将我们的工具传递给Agent，我们只需要将它们格式化为OpenAI工具格式并将它们传递给我们的模型。(通过`bind`函数，我们可以确保每次调用模型时都传入函数。)

In [50]:
llm_with_tools = llm.bind_tools(tools)

### Create the Agent
将这些碎片放在一起，我们现在可以创建Agent。我们将导入两个最后一个实用程序功能：用于格式化中间步骤（Agent操作，工具输出对）的组件，以输入可以发送到模型的输入消息，以及将输出消息转换为Agent操作/Agent饰面的组件。

In [51]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [52]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [54]:
list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The word "eudca" has 5 letters.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_LnrHGcIohbUXNP3rNQa7tDSY', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-677cac1b-9d8e-412b-baa2-389f065086be', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_LnrHGcIohbUXNP3rNQa7tDSY'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"eudca"}', 'id': 'call_LnrHGcIohbUXNP3rNQa7tDSY', 'index': 0}])], tool_call_id='call_LnrHGcIohbUXNP3rNQa7tDSY')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_LnrHGcIohbUXNP3rNQa7tDSY', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata

### Adding memory
太棒了，我们有经纪人了!然而，这个代理是无状态的——它不记得任何关于先前交互的信息。这意味着你不能轻易地问后续问题。让我们通过添加内存来解决这个问题。

为了做到这一点，我们需要做两件事
- 在提示中为内存变量添加一个位置
- 跟踪聊天记录

首先，让我们在提示符中为内存添加一个位置。我们通过为带有`chat_history`键的消息添加占位符来实现这一点。注意，我们将其放在新用户输入的上方(以遵循对话流)。

In [55]:
from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [56]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [57]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [58]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...
"Educa" is not a common English word. It seems to be a variation or abbreviation of the word "education."

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='The word "educa" has 5 letters.')],
 'output': '"Educa" is not a common English word. It seems to be a variation or abbreviation of the word "education."'}

## Streaming
Streaming是LLM应用程序的重要UX考虑因素，代理也不例外。使用代理进行流处理变得更加复杂，因为它不仅是您想要流式传输的最终答案的令牌，而且您可能还想要流式传输代理所采取的中间步骤。

我们的Agent将使用工具API对工具进行调用
1. `where_cat_is_hiding`返回猫藏身的位置
2. `get_items` 列出可以在特定位置找到的项目

这些工具将允许我们在更有趣的情况下探索流，在这种情况下，Agent将不得不使用这两种工具来回答一些问题(例如，回答问题`what items are located where the cat is hiding?`)

In [59]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_core.callbacks import Callbacks
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#### Create the model
注意我们在LLM上设置了`streaming=True`。这将允许我们使用流事件API从Agent中传输令牌。旧版本的LangChain需要这样做。

In [60]:
model = ChatOpenAI(temperature=0, streaming=True)

#### Tools
我们定义了两个工具，它们依赖于聊天模型来生成输出

In [62]:
import random


@tool
async def where_cat_is_hiding() -> str:
    """Where is the cat hiding right now?"""
    return random.choice(["under the bed", "on the shelf"])


@tool
async def get_items(place: str) -> str:
    """Use this tool to look up which items are in the given place."""
    if "bed" in place:  # For under the bed
        return "socks, shoes and dust bunnies"
    if "shelf" in place:  # For 'shelf'
        return "books, penciles and pictures"
    else:  # if the agent decides to ask about a different place
        return "cat snacks"

In [63]:
await where_cat_is_hiding.ainvoke({})

'on the shelf'

In [64]:
await get_items.ainvoke({"place": "shelf"})

'books, penciles and pictures'

#### Initialize the agent
在这里，我们将初始化OpenAI工具代理。

> 请注意，我们使用`"run_name"="Agent"`将`Agent`与我们的`Agent`关联起来。我们将在稍后的`astream_events`API中使用。

In [65]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
print(prompt.messages) #-- to see the prompt


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [66]:
tools = [get_items, where_cat_is_hiding]
agent = create_openai_tools_agent(
    model.with_config({"tags": ["agent_llm"]}), tools, prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools).with_config(
    {"run_name": "Agent"}
)

#### Stream Intermediate Steps
我们将使用`AgentExecutor`的`.stream`方法对代理的中间步骤进行流处理。        
`stream`的输出在(动作、观察)对之间交替进行，如果代理实现了其目标，最后以答案结束。

看起来像这样：
1. actions output
2. observations output
3. actions output
4. observations output
(继续直到达到目标)
然后，如果达到最终目标，Agent将输出最终答案。这些产出的内容总结在这里

输出 | 内容
-- | --
Actions | `actions` `agentaction`或子类，`messages`聊天消息对应动作调用
Observations | `step` 代理到目前为止所做的事情的历史记录，包括当前操作及其观察、带有函数调用结果的`message`聊天消息(也称为观察)
Final answer | `output` `AgentFinish`，`messages`与聊天消息最终输出


In [70]:
# Note: We use `pprint` to print only to depth 1, it makes it easier to see the output from a high level, before digging in.
import pprint

chunks = []

async for chunk in agent_executor.astream(
    {"input": "what's items are located where the cat is hiding?"}
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)

AssertionError: 

#### Using Messages

您可以从输出访问底层消息。在与聊天应用程序一起工作时，使用消息会很好——因为一切都是消息

In [ ]:
chunks[0]["actions"]
for chunk in chunks:
    print(chunk["messages"])

#### Using AgentAction/Observation
输出还在`actions`和`steps`中包含更丰富的结构化信息，这在某些情况下可能很有用，但也可能更难解析。

注意:`AgentFinish`作为流方法的一部分不可用。如果这是你想要添加的东西，请在github上开始讨论并解释为什么需要它。

In [73]:
async for chunk in agent_executor.astream(
    {"input": "what's items are located where the cat is hiding?"}
):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

AssertionError: 

### Custom Streaming With Events
如果Streaming的默认行为不适合您的应用程序，请使用Streaming事件API(例如，如果您需要Streaming来自Agent的单个令牌或工具中发生的表面步骤)。

让我们使用此API来流式传输以下事件：
1. Agent从输入开始
2. Tool从输入开始
3. Tool输出结束
4. 逐个令牌Stream Agent的最终回答
5. Agent结束输出

In [74]:
async for event in agent_executor.astream_events(
    {"input": "where is the cat hiding? what items are in that location?"},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: Agent with input: {'input': 'where is the cat hiding? what items are in that location?'}


AssertionError: 

#### Stream Events from within Tools
如果您的工具利用了LangChain可运行对象(例如，LCEL链，llm，检索器等)，并且您也希望从这些对象中流式处理事件，则需要确保回调被正确传播。

为了了解如何传递回调，让我们重新实现`get_item`工具，使其使用LLM并将回调传递给该LLM。请随意将其调整到您的用例中。

In [75]:
@tool
async def get_items(place: str, callbacks: Callbacks) -> str:  # <--- Accept callbacks
    """Use this tool to look up which items are in the given place."""
    template = ChatPromptTemplate.from_messages(
        [
            (
                "human",
                "Can you tell me what kind of items i might find in the following place: '{place}'. "
                "List at least 3 such items separating them by a comma. And include a brief description of each item..",
            )
        ]
    )
    chain = template | model.with_config(
        {
            "run_name": "Get Items LLM",
            "tags": ["tool_llm"],
            "callbacks": callbacks,  # <-- Propagate callbacks
        }
    )
    chunks = [chunk async for chunk in chain.astream({"place": place})]
    return "".join(chunk.content for chunk in chunks)

接下来，让我们初始化代理，并查看新的输出。

In [76]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
# print(prompt.messages) -- to see the prompt
tools = [get_items, where_cat_is_hiding]
agent = create_openai_tools_agent(
    model.with_config({"tags": ["agent_llm"]}), tools, prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools).with_config(
    {"run_name": "Agent"}
)

async for event in agent_executor.astream_events(
    {"input": "where is the cat hiding? what items are in that location?"},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: Agent with input: {'input': 'where is the cat hiding? what items are in that location?'}


AssertionError: 

#### Other aproaches
Using `astream_log`

> 注意:你也可以使用`astream_log`API。该API生成执行期间发生的所有事件的细粒度日志。日志格式基于JSONPatch标准。它是粒度的，但是需要努力去解析。出于这个原因，我们创建了`astream_events `API。

In [77]:
i = 0
async for chunk in agent_executor.astream_log(
    {"input": "where is the cat hiding? what items are in that location?"},
):
    print(chunk)
    i += 1
    if i > 10:
        break

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'ea35335b-3df7-4341-9234-e314539b4f64',
            'logs': {},
            'name': 'Agent',
            'streamed_output': [],
            'type': 'chain'}})
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableSequence',
  'value': {'end_time': None,
            'final_output': None,
            'id': '91c6b9e5-e383-4ad0-99f1-b99ae7a8633c',
            'metadata': {},
            'name': 'RunnableSequence',
            'start_time': '2024-05-23T09:20:41.946+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': [],
            'type': 'chain'}})
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableAssign<agent_scratchpad>',
  'value': {'end_time': None,
            'final_output': None,
            'id': '713978ba-8a78-4214-9092-05333a26a581',
            'metadata': {},
            'name': 'RunnableAssign<agent_scratchpad>',
            'start_ti

这可能需要一些逻辑来获得一个可行的格式

In [78]:
i = 0
path_status = {}
async for chunk in agent_executor.astream_log(
    {"input": "where is the cat hiding? what items are in that location?"},
):
    for op in chunk.ops:
        if op["op"] == "add":
            if op["path"] not in path_status:
                path_status[op["path"]] = op["value"]
            else:
                path_status[op["path"]] += op["value"]
    print(op["path"])
    print(path_status.get(op["path"]))
    print("----")
    i += 1
    if i > 30:
        break


None
----
/logs/RunnableSequence
{'id': '08193882-b394-47b8-b03b-32063d836694', 'name': 'RunnableSequence', 'type': 'chain', 'tags': [], 'metadata': {}, 'start_time': '2024-05-23T09:21:14.498+00:00', 'streamed_output': [], 'streamed_output_str': [], 'final_output': None, 'end_time': None}
----
/logs/RunnableAssign<agent_scratchpad>
{'id': 'b58ea2b0-77e7-4bcb-b687-ce3f251962b8', 'name': 'RunnableAssign<agent_scratchpad>', 'type': 'chain', 'tags': ['seq:step:1'], 'metadata': {}, 'start_time': '2024-05-23T09:21:14.507+00:00', 'streamed_output': [], 'streamed_output_str': [], 'final_output': None, 'end_time': None}
----
/logs/RunnableAssign<agent_scratchpad>/streamed_output/-
{'input': 'where is the cat hiding? what items are in that location?', 'intermediate_steps': []}
----
/logs/RunnableParallel<agent_scratchpad>
{'id': '542efb1f-1758-4cbb-81a5-3b503e52279b', 'name': 'RunnableParallel<agent_scratchpad>', 'type': 'chain', 'tags': [], 'metadata': {}, 'start_time': '2024-05-23T09:21:14.51

AssertionError: 

Using `callbacks (Legacy)`

streaming 的另一种方法是使用回调。如果您还在使用旧版本的LangChain并且无法升级，这可能会很有用。

一般来说，这不是一个推荐的方法，因为
1. 对于大多数应用程序，您需要创建两个worker，将回调写入队列，并让另一个worker从队列中读取(即，实现此工作存在隐藏的复杂性)。
2. 结束事件可能缺少一些元数据(例如，运行名称)。因此，如果您需要额外的元数据，您应该从BaseTracer而不是AsyncCallbackHandler继承，以从运行(也称为跟踪)中获取相关信息，或者根据运行id自己实现聚合逻辑。
3. 有不一致的行为与回调(例如，如何输入和输出编码)取决于你需要解决的回调类型。

为了便于说明，我们在下面实现了一个回调，该回调显示了如何逐个令牌获取流。您可以根据应用程序的需要随意实现其他回调。

In [ ]:
from typing import TYPE_CHECKING, Any, Dict, List, Optional, Sequence, TypeVar, Union
from uuid import UUID

from langchain_core.callbacks.base import AsyncCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import ChatGenerationChunk, GenerationChunk, LLMResult

# Here is a custom handler that will print the tokens to stdout.
# Instead of printing to stdout you can send the data elsewhere; e.g., to a streaming API response


class TokenByTokenHandler(AsyncCallbackHandler):
    def __init__(self, tags_of_interest: List[str]) -> None:
        """A custom call back handler.

        Args:
            tags_of_interest: Only LLM tokens from models with these tags will be
                              printed.
        """
        self.tags_of_interest = tags_of_interest

    async def on_chain_start(
        self,
        serialized: Dict[str, Any],
        inputs: Dict[str, Any],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        """Run when chain starts running."""
        print("on chain start: ")
        print(inputs)

    async def on_chain_end(
        self,
        outputs: Dict[str, Any],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> None:
        """Run when chain ends running."""
        print("On chain end")
        print(outputs)

    async def on_chat_model_start(
        self,
        serialized: Dict[str, Any],
        messages: List[List[BaseMessage]],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> Any:
        """Run when a chat model starts running."""
        overlap_tags = self.get_overlap_tags(tags)

        if overlap_tags:
            print(",".join(overlap_tags), end=": ", flush=True)

    def on_tool_start(
        self,
        serialized: Dict[str, Any],
        input_str: str,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        inputs: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> Any:
        """Run when tool starts running."""
        print("Tool start")
        print(serialized)

    def on_tool_end(
        self,
        output: Any,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> Any:
        """Run when tool ends running."""
        print("Tool end")
        print(str(output))

    async def on_llm_end(
        self,
        response: LLMResult,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> None:
        """Run when LLM ends running."""
        overlap_tags = self.get_overlap_tags(tags)

        if overlap_tags:
            # Who can argue with beauty?
            print()
            print()

    def get_overlap_tags(self, tags: Optional[List[str]]) -> List[str]:
        """Check for overlap with filtered tags."""
        if not tags:
            return []
        return sorted(set(tags or []) & set(self.tags_of_interest or []))

    async def on_llm_new_token(
        self,
        token: str,
        *,
        chunk: Optional[Union[GenerationChunk, ChatGenerationChunk]] = None,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> None:
        """Run on new LLM token. Only available when streaming is enabled."""
        overlap_tags = self.get_overlap_tags(tags)

        if token and overlap_tags:
            print(token, end="|", flush=True)


handler = TokenByTokenHandler(tags_of_interest=["tool_llm", "agent_llm"])

result = await agent_executor.ainvoke(
    {"input": "where is the cat hiding and what items can be found there?"},
    {"callbacks": [handler]},
)

#### 将Agent作为迭代器运行
可以将代理作为迭代器运行，以便根据需要添加人工循环检查。

